In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
%matplotlib inline
import pandas as pd
%ls Data

high.csv  highp_finalresults.csv  low.csv  lowp_finalresults.csv


# Data Formatting
Here we give format to data for better manipulation.

Few things are important to state about the following process of formatting:

### Mass parameters were passed to logarithmic scale as it is a usual practice in astrophysics for a better analysis of various relations. 

### Planetary parameters [a(i),emegas(i),rplanet(i)/radtie] were omitted as they are not necessary to calculate quantities to predict, nor they are initial conditions of the system (we want to predict).

### Parameters ['ident', 't', 'gama', 'apert', 'fpert', 'constmigI'] were dropped as well. Identity index is an arbitrary identifier and is not taken into account. The rest of omitted parameters are the same for all systems and have no prediction power. For these parameters, we calculated the standard deviation and verified that it is zero. (Data_Formatting.ipynb).


----------------------------

----------------------------

In [9]:
# Loads data (Fills NaN values as 0 for simplicity) THERE IS ONLY ONE NaN VALUE 
data=pd.read_csv("./Data/highp_finalresults.csv",sep = ',',
                 engine = 'python',skipinitialspace = True,na_values = 'NaN ').fillna(0)
# Each row, (for now is a different planet)
# Adds column named n_planet___ it is later going to be summed
data["n_planets"] = 1

# Selects useful planet data only to produce: number of planets, total mass of planets. 
# Identity of system is conserved
planet_data = data[['ident','emepla(i)/emet','n_planets']]

# Selects system data only. 
# System columns corresponding to initial condition
# According to repository README.md, we have to drop: a(i),emegas(i),emepla(i)/emed,rplanet(i)/radtie
system_data = data.drop(['a(i)','emegas(i)','emepla(i)/emet','rplanet(i)/radtie','n_planets'],1)

# System data is repeated as rows represent planets -> so we take the mean
system_data = system_data.groupby(system_data['ident']).mean()

# From planets data it is only necesary the total data (total mass, total number of planets)
planet_data = planet_data.groupby(planet_data['ident']).sum()

# Concatenate two data frames to obtain final formatted data
data = pd.concat([system_data,planet_data], axis=1).reset_index()

# Drops unimportant parameters with null std()
# 0,2,8,9,10,11 are unimportant parameters
# Data for the first point
print(data.keys()[[0,2,8,9,10,11]])
data = data.drop(data.keys()[[0,2,8,9,10,11]],1)

# Creates column corresponding to mass efficiency
data['m_efficiency'] = data['emepla(i)/emet']/data['emed']

# Logarithmic scales for mass-related parameters (1e-5 to avoid log(0)=-infty)
data['emepla(i)/emet'] = np.log(data['emepla(i)/emet']+1e-5)
data['emestar'] = np.log(data['emestar']+1e-5)
data['emed'] = np.log(data['emed']+1e-5)
#data['sigmag_0'] = np.log(data['sigmag_0']+1e-5)

# Important position of parameters
print(' total mass, number of planets, mass efficiency',np.where(data.keys()=='n_planets'),
      np.where(data.keys()=='emepla(i)/emet'),np.where(data.keys()=='m_efficiency'))

# Shows variable data
data.head(15)

Index(['ident', 't', 'gama', 'apert', 'fpert', 'constmigI'], dtype='object')
number of planets, total mass, mass efficiency (array([9]),) (array([8]),) (array([10]),)


,it,emestar,rc,qest,sigmag_0,emed,emetal,taugas,emepla(i)/emet,n_planets,m_efficiency
0,199993,75.157068,59.882370,1.110726,51.287674,-2.040144,0.077070,9.268935e+06,2.096372,11,62.589101
1,198840,74.422324,40.040195,3.201758,16.765976,-3.962790,-0.092791,3.525594e+06,1.039386,13,148.814199
2,199995,75.360733,40.647163,1.965345,67.644768,-2.538181,0.514770,9.390406e+06,2.076882,9,101.006870
3,199755,74.967751,29.539343,9.568532,16.213096,-4.604171,0.328146,1.644711e+06,1.938003,18,694.485525
4,199319,74.742551,61.195038,2.230675,15.866622,-3.169848,-0.220031,9.468353e+06,1.161167,10,76.039252
5,199834,75.485455,38.977428,5.082291,31.660674,-3.381101,-0.150160,1.014449e+06,1.539754,19,137.159726
6,198714,74.912779,47.343380,5.121542,12.623480,-3.911523,0.021273,1.790533e+06,1.220502,17,169.443990
7,199619,74.853960,42.739063,3.225643,22.460303,-3.540115,-0.092571,2.721395e+06,1.674280,12,183.963532
8,198489,75.488793,58.158928,4.369815,18.402952,-3.123338,-0.282408,6.017040e+06,1.385713,12,90.856065
9,199991,75.323966,30.851170,1.224490,163.499726,-2.207184,-0.065073,1.576468e+06,2.360289,11,96.309095


# The Regression

We chose nearest neighbor method to predict the required parameters given some training data.

This algorithm is similar to that analogue of classification where nearest neighbors 'vote' to decide the label of a poind determined by X. Here, an average (it can be weighted) is taken to produce the continuous output for the prediction variable.

For each of the wanted predictions: (number of planets, total mass, mass efficiency), this method is going to be applied. But first, it is going to be chosen the optimal parameters.

The parameters we are going to vary are just the weight function, which can be 'uniform' (each neighbor has the same power of decision on the average) or 'distance' (nearest neighbors have more weight than farther neighbors); and the number of neighbors to have into account

To decide which method is more efficient, we use the usual exponential of $\chi^2$ as efficiency function:

$$ \varepsilon = exp\left(-\frac{1}{2}\frac{\sum_{i=1}^N (y_i^*-y_i)^2}{N\sigma_y^2}\right)$$

Here $y^*_i$ is the prediced value and $y_i$ is the real value. $N$ is the number of data and $\sigma_y$ is the standard deviation of the data set $\{y_i\}$. This efficiency $\epsilon$ has the property that is scale-invariant and does not (in general) increases with the size of the data set $\{y_i\}$.

# Number of planets prediction
# Number of planets prediction
# Number of planets prediction


In [ ]:
# Non initial condition parameters are dropped (total mass, mass efficiency)
Xdata =  data[data.keys()[:9]]
Ydata = 